In [7]:
import os
from textwrap import dedent

import litellm
from gait import s_message, u_message
from pydantic import BaseModel, Field
from rich.pretty import pprint

In [8]:
class RouteModel(BaseModel):
    oper: str = Field(
        ...,
        description="The operation. Can be either ROUTE, GEOCODE or TEMPERATURE",
    )

In [9]:
system = dedent(
    """
You are an expert at generating simple JSON documents that recommend one of three possible solutions.

A possible >>>SOLUTION<<< is either:

1. ROUTE
2. GEOCODE
3. TEMPERATURE

ROUTE is when you have to route between two addresses, places or cities on a road network.
GEOCODE is when you have to convert an address, city, state, zip code or country to an x,y point.
TEMPERATURE is when you have to tell the temperature or tell the weather for any place on earth.

Make sure to:

1. Not generate ANY code in the answer.
2. Only output a simple JSON document like {"oper":>>>SOLUTION<<<}
3. Only recommend ONE of the possible answers.
4. Infer and learn the correct solution based on the input.
"""
).strip()

In [10]:
messages = [
    s_message(system),
    # u_message("Show the route to San Diego from San Francisco"),
    u_message("What is the location of Florence, Italy?"),
    # u_message("What is the current temperature of Florence, Italy?"),
]

In [11]:
response = litellm.completion(
    # model="ollama_chat/phi4:14b-q8_0",
    model="azure/gpt-4.1-mini",
    api_base=os.environ["AZURE_API_URL"] + "/gpt-4.1-mini",
    response_format=RouteModel,
    messages=messages,
    temperature=0.0,
    top_p=1.0,
    n=1,
)

In [12]:
pprint(
    response.choices[0],
    expand_all=True,
)

Choices(
│   finish_reason='stop',
│   index=0,
│   message=Message(
│   │   content='{"oper":"GEOCODE"}',
│   │   role='assistant',
│   │   tool_calls=None,
│   │   function_call=None,
│   │   provider_specific_fields={
│   │   │   'refusal': None
│   │   }
│   )
)